In [4]:
%cd /scratch/mt/ashapiro/Hate_Speech/Multitask_trial/

/scratch/mt/ashapiro/Hate_Speech/Multitask_trial


In [8]:
%ls Data/multitask_data/

train_48.csv  train_lhsab.csv  train_thsab.csv


In [5]:
import numpy as np
import torch
import torch.nn as nn
import transformers
import nlp
import logging
from datasets import load_dataset
from model import * 
logging.basicConfig(level=logging.INFO)

# All

## Preparing Data

In [19]:
task_names = ['offensive', 'hatespeech', 'forty_eight','lhsab', 'thsab']

In [9]:
dataset_dict = {
    "offensive": load_dataset("csv", data_files={'train': "Data/train/trainA_prepro_large.csv", 'test': "Data/test/testA_prepro.csv" } ),
    "hatespeech": load_dataset("csv", data_files={'train': "Data/train/trainB_prepro_large.csv", 'test': "Data/test/testB_prepro.csv" } ),
    "forty_eight": load_dataset("csv", data_files={'train': "Data/multitask_data/train_48.csv" } ),
    "lhsab": load_dataset("csv", data_files={'train': "Data/multitask_data/train_lhsab.csv"} ),
    "thsab": load_dataset("csv", data_files={'train': "Data/multitask_data/train_thsab.csv" } )    
}

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 74.86it/s]


Dataset csv downloaded and prepared to /home/ashapiro/.cache/huggingface/datasets/csv/default-2328e34458fcd88e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 278.70it/s]


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 84.28it/s]


Dataset csv downloaded and prepared to /home/ashapiro/.cache/huggingface/datasets/csv/default-6be373aac13c1a94/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


...

Dataset csv downloaded and prepared to /home/ashapiro/.cache/huggingface/datasets/csv/default-71d561b6517dac88/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 408.28it/s]


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 73.91it/s]


Dataset csv downloaded and prepared to /home/ashapiro/.cache/huggingface/datasets/csv/default-b9870a1789795d7b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 318.62it/s]


## Setting Model

In [11]:
model_name = "/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/"
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        "offensive": transformers.AutoModelForSequenceClassification,
        "hatespeech": transformers.AutoModelForSequenceClassification,
        "forty_eight": transformers.AutoModelForSequenceClassification,
        "lhsab": transformers.AutoModelForSequenceClassification,
        "thsab": transformers.AutoModelForSequenceClassification
        
    },
    model_config_dict={
        "offensive": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "hatespeech": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "forty_eight": transformers.AutoConfig.from_pretrained(model_name, num_labels=3),
        "lhsab": transformers.AutoConfig.from_pretrained(model_name, num_labels=3),
        "thsab": transformers.AutoConfig.from_pretrained(model_name, num_labels=3)
    },
)

In [12]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [13]:
max_length = 512

def convert_to_features(example_batch):
    inputs = list(example_batch['text'])
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["labels"]
    return features

convert_func_dict = {
    "offensive": convert_to_features,
    "hatespeech": convert_to_features,
    "forty_eight": convert_to_features,
    "lhsab": convert_to_features,
    "thsab": convert_to_features,
}

In [14]:
columns_dict = {
    "offensive": ['input_ids', 'attention_mask', 'labels'],
    "hatespeech": ['input_ids', 'attention_mask', 'labels'],
    "forty_eight": ['input_ids', 'attention_mask', 'labels'],
    "lhsab": ['input_ids', 'attention_mask', 'labels'],
    "thsab": ['input_ids', 'attention_mask', 'labels'],
}

features_dict = {}
for task_name, dataset in dataset_dict.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
        features_dict[task_name][phase] = phase_dataset.map(
            convert_func_dict[task_name],
            batched=True,
            load_from_cache_file=False,
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        features_dict[task_name][phase].set_format(
            type="torch", 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

100%|██████████| 20/20 [00:08<00:00,  2.47ba/s]


offensive train 19906 19906
offensive train 19906 19906


100%|██████████| 2/2 [00:00<00:00,  4.01ba/s]


offensive test 1270 1270
offensive test 1270 1270


100%|██████████| 5/5 [00:02<00:00,  2.31ba/s]


...

forty_eight train 1359 1359
forty_eight train 1359 1359


100%|██████████| 11/11 [00:04<00:00,  2.71ba/s]


lhsab train 10950 10950
lhsab train 10950 10950


100%|██████████| 12/12 [00:04<00:00,  2.83ba/s]


thsab train 11460 11460
thsab train 11460 11460


In [15]:
train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in features_dict.items()
}
args = transformers.TrainingArguments(
        output_dir="./models/multitask_model/5_main_tasks/4_epochs/",
        overwrite_output_dir=True,
        learning_rate=2e-5,
        do_train=True,
        num_train_epochs=4,
        # Adjust batch size if this doesn't fit on the Colab GPU
        per_device_train_batch_size=16,  
        save_steps=3000,)

trainer = MultitaskTrainer(
    model=multitask_model,
    args=args,
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)

wandb: Currently logged in as: ahmadshapiro (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [16]:
trainer.train()

Iteration:  16%|█▋        | 500/3032 [04:30<25:39,  1.64it/s]

{"loss": 0.6605726473927498, "learning_rate": 1.9175461741424804e-05, "epoch": 0.16490765171503957, "step": 500}



Iteration:  33%|███▎      | 1000/3032 [09:01<19:51,  1.71it/s]

{"loss": 0.5118207466900349, "learning_rate": 1.8350923482849604e-05, "epoch": 0.32981530343007914, "step": 1000}



Iteration:  49%|████▉     | 1500/3032 [13:32<15:00,  1.70it/s]

...

{"loss": 0.06983707585709635, "learning_rate": 1.0356200527704487e-06, "epoch": 3.7928759894459105, "step": 11500}



Iteration:  96%|█████████▌| 2903/3032 [25:48<01:08,  1.88it/s]

{"loss": 0.07198260226415004, "learning_rate": 2.1108179419525068e-07, "epoch": 3.9577836411609497, "step": 12000}



Epoch: 100%|██████████| 4/4 [1:49:00<00:00, 1635.15s/it]


TrainOutput(global_step=12128, training_loss=0.21598201864807112)

In [17]:
def evaluate(trainer, features_dict, epochs):
    from sklearn.metrics import classification_report
    preds_dict = {}
    for task_name in task_names:
        eval_dataloader = DataLoaderWithTaskname(
            task_name,
            trainer.get_eval_dataloader(eval_dataset=features_dict[task_name]["test"])
        )
        print(eval_dataloader.data_loader.collate_fn)
        preds_dict[task_name] = trainer._prediction_loop(
            eval_dataloader, 
            description=f"Test: {task_name}",
        )
        print(f"Classification Report for Task {task_name} trained for {epochs} epochs")
        print(classification_report(y_pred=np.argmax(preds_dict[task_name].predictions, axis=1),  y_true=preds_dict[task_name].label_ids, digits=16))

In [20]:
evaluate(trainer, features_dict, epochs=4)

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff56df1990>>


Test: offensive: 100%|██████████| 159/159 [00:13<00:00, 12.06it/s]


Classification Report for Task offensive trained for 4 epochs
                  precision    recall  f1-score   support

               0  0.9110576923076923 0.8752886836027713 0.8928150765606596       866
               1  0.7534246575342466 0.8168316831683168 0.7838479809976246       404

        accuracy                      0.8566929133858268      1270
       macro avg  0.8322411749209695 0.8460601833855441 0.8383315287791422      1270
    weighted avg  0.8609130103797615 0.8566929133858268 0.8581515280508437      1270

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff56df1990>>


Test: hatespeech: 100%|██████████| 159/159 [00:13<00:00, 12.18it/s]

Classification Report for Task hatespeech trained for 4 epochs
                  precision    recall  f1-score   support

               0  0.9884615384615385 0.8854435831180018 0.9341208541572013      1161
               1  0.4217391304347826 0.8899082568807339 0.5722713864306785       109

        accuracy                      0.8858267716535433      1270
       macro avg  0.7051003344481606 0.8876759199993678 0.7531961202939399      1270
    weighted avg  0.9398215837568801 0.8858267716535433 0.9030644825176808      1270



KeyError: 'test'

#### Lets see for two more epoch

In [22]:
trainer.args.num_train_epochs = 2

In [ ]:
trainer.train()

Iteration:  16%|█▋        | 499/3032 [04:26<22:24,  1.88it/s]wandb: WARNING Step must only increase in log calls.  Step 500 < 12000; dropping {'loss': 0.09856156264507444, 'learning_rate': 1.8350923482849604e-05, 'epoch': 0.16490765171503957}.

Iteration:  16%|█▋        | 500/3032 [04:27<22:36,  1.87it/s]

{"loss": 0.09856156264507444, "learning_rate": 1.8350923482849604e-05, "epoch": 0.16490765171503957, "step": 500}



Iteration:  33%|███▎      | 999/3032 [08:54<17:36,  1.92it/s]wandb: WARNING Step must only increase in log calls.  Step 1000 < 12000; dropping {'loss': 0.11112148779537528, 'learning_rate': 1.670184696569921e-05, 'epoch': 0.32981530343007914}.

Iteration:  33%|███▎      | 1000/3032 [08:54<17:41,  1.91it/s]

{"loss": 0.11112148779537528, "learning_rate": 1.670184696569921e-05, "epoch": 0.32981530343007914, "step": 1000}



Iteration:  49%|████▉     | 1499/3032 [13:21<13:37,  1.87it/s]wandb: WARNING Step must only increase in log calls.  Step 1500 < 12000; dropping {'loss': 0.11249433731511817, 'learning_rate': 1.5052770448548815e-05, 'epoch': 0.4947229551451187}.

Iteration:  49%|████▉     | 1500/3032 [13:22<13:39,  1.87it/s]

...


Iteration:  15%|█▌        | 467/3032 [04:08<23:01,  1.86it/s]wandb: WARNING Step must only increase in log calls.  Step 3500 < 12000; dropping {'loss': 0.05659188774196082, 'learning_rate': 8.456464379947231e-06, 'epoch': 1.154353562005277}.

Iteration:  15%|█▌        | 468/3032 [04:09<22:55,  1.86it/s]

{"loss": 0.05659188774196082, "learning_rate": 8.456464379947231e-06, "epoch": 1.154353562005277, "step": 3500}



Iteration:  22%|██▏       | 662/3032 [05:52<21:14,  1.86it/s]wandb: Network error resolved after 0:03:11.042432, resuming normal operation.

Iteration:  32%|███▏      | 967/3032 [08:29<17:22,  1.98it/s]wandb: WARNING Step must only increase in log calls.  Step 4000 < 12000; dropping {'loss': 0.05963085182575742, 'learning_rate': 6.807387862796835e-06, 'epoch': 1.3192612137203166}.

Iteration:  32%|███▏      | 968/3032 [08:29<17:17,  1.99it/s]

{"loss": 0.05963085182575742, "learning_rate": 6.807387862796835e-06, "epoch": 1.3192612137203166, "step": 4000}



Iteration:  34%|███▍      | 1041/3032 [09:06<16:51,  1.97it/s]

In [33]:
evaluate(trainer, features_dict, epochs=5)

Test: offensive:   1%|          | 1/159 [00:00<00:16,  9.84it/s]

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech:   1%|▏         | 2/159 [00:00<00:12, 12.20it/s]

Classification Report for Task offensive trained for 5 epochs
                  precision    recall  f1-score   support

               0  0.9061371841155235 0.8695150115473441 0.8874484384207424       866
               1  0.7425968109339408 0.8069306930693070 0.7734282325029657       404

        accuracy                      0.8496062992125984      1270
       macro avg  0.8243669975247321 0.8382228523083255 0.8304383354618541      1270
    weighted avg  0.8541133173711460 0.8496062992125984 0.8511774437823314      1270

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech_classes:   1%|▏         | 2/159 [00:00<00:13, 12.04it/s]

Classification Report for Task hatespeech trained for 5 epochs
                  precision    recall  f1-score   support

               0  0.9731136166522116 0.9664082687338501 0.9697493517718236      1161
               1  0.6666666666666666 0.7155963302752294 0.6902654867256638       109

        accuracy                      0.9448818897637795      1270
       macro avg  0.8198901416594391 0.8410022995045398 0.8300074192487437      1270
    weighted avg  0.9468122642518775 0.9448818897637795 0.9457621539056573      1270

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech_classes: 100%|██████████| 159/159 [00:13<00:00, 11.96it/s]

Classification Report for Task hatespeech_classes trained for 5 epochs
                  precision    recall  f1-score   support

               0  0.9699054170249355 0.9715762273901809 0.9707401032702238      1161
               1  0.6666666666666666 0.5714285714285714 0.6153846153846153        28
               2  0.0000000000000000 0.0000000000000000 0.0000000000000000         4
               3  0.5333333333333333 0.5714285714285714 0.5517241379310344        14
               4  0.0000000000000000 0.0000000000000000 0.0000000000000000         1
               5  0.1538461538461539 0.2000000000000000 0.1739130434782609        10
               6  0.6909090909090909 0.7307692307692307 0.7102803738317757        52

        accuracy                      0.9385826771653544      1270
       macro avg  0.4306658088257400 0.4350289430023649 0.4317203248422729      1270
    weighted avg  0.9367395722559195 0.9385826771653544 0.9375258873484790      1270




/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this

### 2 Epochs

In [18]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8418723545933512}

In [19]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8371149406524729}

In [20]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.3926597611174843}

### 3 Epochs

In [21]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8418723545933512}

In [27]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8325985296056062}

In [28]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4176057674898591}

### 4 Epochs

In [25]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8469622661091527}

In [27]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8325985296056062}

In [28]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4176057674898591}

### 5 Epochs

In [43]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8387301587301587}

In [44]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8107379458902904}

In [45]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4355893433350976}

# Combo

## Preparing Data

In [4]:
task_names = ['offensive', 'hatespeech', 'hatespeech_classes']

In [5]:
dataset_dict = {
    "offensive": load_dataset("csv", data_files={'train': "Data/train/trainA_prepro_combo.csv", 'test': "Data/test/testA_prepro.csv" } ),
    "hatespeech": load_dataset("csv", data_files={'train': "Data/train/trainB_prepro_combo.csv", 'test': "Data/test/testB_prepro.csv" } ),
    "hatespeech_classes": load_dataset("csv", data_files={'train': "Data/train/trainC_prepro.csv", 'test': "Data/test/testC_prepro.csv" } ),
}

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 69.37it/s]


Dataset csv downloaded and prepared to /home/ashapiro/.cache/huggingface/datasets/csv/default-1337ce598b95f03a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 571.86it/s]


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 72.30it/s]


Dataset csv downloaded and prepared to /home/ashapiro/.cache/huggingface/datasets/csv/default-ea8e0edd0e0f031f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 578.56it/s]


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 77.70it/s]


Dataset csv downloaded and prepared to /home/ashapiro/.cache/huggingface/datasets/csv/default-0210a04391128139/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 331.76it/s]


## Setting Model

In [6]:
model_name = "/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/"
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        "offensive": transformers.AutoModelForSequenceClassification,
        "hatespeech": transformers.AutoModelForSequenceClassification,
        "hatespeech_classes": transformers.AutoModelForSequenceClassification,
    },
    model_config_dict={
        "offensive": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "hatespeech": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "hatespeech_classes": transformers.AutoConfig.from_pretrained(model_name, num_labels=7),
    },
)

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [8]:
max_length = 512

def convert_to_features(example_batch):
    inputs = list(example_batch['text'])
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["labels"]
    return features

convert_func_dict = {
    "offensive": convert_to_features,
    "hatespeech": convert_to_features,
    "hatespeech_classes": convert_to_features,
}

In [9]:
columns_dict = {
    "offensive": ['input_ids', 'attention_mask', 'labels'],
    "hatespeech": ['input_ids', 'attention_mask', 'labels'],
    "hatespeech_classes": ['input_ids', 'attention_mask', 'labels'],
}

features_dict = {}
for task_name, dataset in dataset_dict.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
        features_dict[task_name][phase] = phase_dataset.map(
            convert_func_dict[task_name],
            batched=True,
            load_from_cache_file=False,
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        features_dict[task_name][phase].set_format(
            type="torch", 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

100%|██████████| 42/42 [00:19<00:00,  2.21ba/s]


offensive train 41212 41212
offensive train 41212 41212


100%|██████████| 2/2 [00:00<00:00,  3.99ba/s]


offensive test 1270 1270
offensive test 1270 1270


100%|██████████| 33/33 [00:15<00:00,  2.13ba/s]


...

hatespeech test 1270 1270
hatespeech test 1270 1270


100%|██████████| 9/9 [00:03<00:00,  2.53ba/s]


hatespeech_classes train 8887 8887
hatespeech_classes train 8887 8887


100%|██████████| 2/2 [00:00<00:00,  3.96ba/s]

hatespeech_classes test 1270 1270
hatespeech_classes test 1270 1270


In [10]:
eval_dataset = {
    task_name: dataset["test"] 
    for task_name, dataset in features_dict.items()
}

In [11]:
train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in features_dict.items()
}
args = transformers.TrainingArguments(
        output_dir="./models/multitask_model/3_main_tasks/combo_data/4_epochs/",
        overwrite_output_dir=True,
        learning_rate=2e-5,
        do_train=True,
        num_train_epochs=4,
        # Adjust batch size if this doesn't fit on the Colab GPU
        per_device_train_batch_size=16,  
        save_steps=3000,)

trainer = MultitaskTrainer(
    model=multitask_model,
    args=args,
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

Iteration:   2%|▏         | 89/5172 [00:48<46:13,  1.83it/s]

In [13]:
task_names = ['offensive','hatespeech','hatespeech_classes']

In [32]:
def evaluate(trainer, features_dict, epochs):
    from sklearn.metrics import classification_report
    preds_dict = {}
    for task_name in task_names:
        eval_dataloader = DataLoaderWithTaskname(
            task_name,
            trainer.get_eval_dataloader(eval_dataset=features_dict[task_name]["test"])
        )
        print(eval_dataloader.data_loader.collate_fn)
        preds_dict[task_name] = trainer._prediction_loop(
            eval_dataloader, 
            description=f"Test: {task_name}",
        )
        print(f"Classification Report for Task {task_name} trained for {epochs} epochs")
        print(classification_report(y_pred=np.argmax(preds_dict[task_name].predictions, axis=1),  y_true=preds_dict[task_name].label_ids, digits=16))

In [26]:
evaluate(trainer, features_dict, epochs=4)

Test: offensive:   1%|          | 1/159 [00:00<00:16,  9.79it/s]

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech:   1%|▏         | 2/159 [00:00<00:13, 11.60it/s]

Classification Report for Task offensive trained for 4 epochs
              precision    recall  f1-score   support

           0       0.92      0.86      0.89       866
           1       0.73      0.83      0.78       404

    accuracy                           0.85      1270
   macro avg       0.82      0.84      0.83      1270
weighted avg       0.86      0.85      0.85      1270

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech_classes:   1%|▏         | 2/159 [00:00<00:13, 12.02it/s]

Classification Report for Task hatespeech trained for 4 epochs
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1161
           1       0.65      0.75      0.70       109

    accuracy                           0.94      1270
   macro avg       0.81      0.86      0.83      1270
weighted avg       0.95      0.94      0.95      1270

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech_classes: 100%|██████████| 159/159 [00:13<00:00, 11.93it/s]

Classification Report for Task hatespeech_classes trained for 4 epochs
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1161
           1       0.58      0.64      0.61        28
           2       0.00      0.00      0.00         4
           3       0.56      0.64      0.60        14
           4       0.00      0.00      0.00         1
           5       0.20      0.10      0.13        10
           6       0.67      0.77      0.71        52

    accuracy                           0.94      1270
   macro avg       0.43      0.45      0.43      1270
weighted avg       0.94      0.94      0.94      1270




/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this

#### Lets see for one more epoch

In [27]:
trainer.args.num_train_epochs = 1

In [28]:
trainer.train()

Iteration:  24%|██▍       | 499/2101 [04:28<14:26,  1.85it/s]wandb: WARNING Step must only increase in log calls.  Step 500 < 8000; dropping {'loss': 0.12110608770386898, 'learning_rate': 1.524036173250833e-05, 'epoch': 0.23798191337458352}.

Iteration:  24%|██▍       | 500/2101 [04:29<14:27,  1.85it/s]

{"loss": 0.12110608770386898, "learning_rate": 1.524036173250833e-05, "epoch": 0.23798191337458352, "step": 500}



Iteration:  48%|████▊     | 999/2101 [08:58<09:58,  1.84it/s]wandb: WARNING Step must only increase in log calls.  Step 1000 < 8000; dropping {'loss': 0.12189705061679706, 'learning_rate': 1.048072346501666e-05, 'epoch': 0.47596382674916704}.

Iteration:  48%|████▊     | 1000/2101 [08:59<09:59,  1.84it/s]

{"loss": 0.12189705061679706, "learning_rate": 1.048072346501666e-05, "epoch": 0.47596382674916704, "step": 1000}



Iteration:  71%|███████▏  | 1499/2101 [13:26<05:25,  1.85it/s]wandb: WARNING Step must only increase in log calls.  Step 1500 < 8000; dropping {'loss': 0.09711660955034312, 'learning_rate': 5.721085197524988e-06, 'epoch': 0.7139457401237506}.

Iteration:  71%|███████▏  | 1500/2101 [13:27<05:24,  1.85it/s]

{"loss": 0.09711660955034312, "learning_rate": 5.721085197524988e-06, "epoch": 0.7139457401237506, "step": 1500}



Iteration:  95%|█████████▌| 1999/2101 [17:55<00:53,  1.91it/s]wandb: WARNING Step must only increase in log calls.  Step 2000 < 8000; dropping {'loss': 0.10913289758379688, 'learning_rate': 9.614469300333174e-07, 'epoch': 0.9519276534983341}.

Iteration:  95%|█████████▌| 2000/2101 [17:56<00:52,  1.94it/s]

{"loss": 0.10913289758379688, "learning_rate": 9.614469300333174e-07, "epoch": 0.9519276534983341, "step": 2000}



Epoch: 100%|██████████| 1/1 [18:50<00:00, 1130.53s/it]


TrainOutput(global_step=2101, training_loss=0.11203017262371649)

In [33]:
evaluate(trainer, features_dict, epochs=5)

Test: offensive:   1%|          | 1/159 [00:00<00:16,  9.84it/s]

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech:   1%|▏         | 2/159 [00:00<00:12, 12.20it/s]

Classification Report for Task offensive trained for 5 epochs
                  precision    recall  f1-score   support

               0  0.9061371841155235 0.8695150115473441 0.8874484384207424       866
               1  0.7425968109339408 0.8069306930693070 0.7734282325029657       404

        accuracy                      0.8496062992125984      1270
       macro avg  0.8243669975247321 0.8382228523083255 0.8304383354618541      1270
    weighted avg  0.8541133173711460 0.8496062992125984 0.8511774437823314      1270

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech_classes:   1%|▏         | 2/159 [00:00<00:13, 12.04it/s]

Classification Report for Task hatespeech trained for 5 epochs
                  precision    recall  f1-score   support

               0  0.9731136166522116 0.9664082687338501 0.9697493517718236      1161
               1  0.6666666666666666 0.7155963302752294 0.6902654867256638       109

        accuracy                      0.9448818897637795      1270
       macro avg  0.8198901416594391 0.8410022995045398 0.8300074192487437      1270
    weighted avg  0.9468122642518775 0.9448818897637795 0.9457621539056573      1270

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7fff8edc4ee0>>


Test: hatespeech_classes: 100%|██████████| 159/159 [00:13<00:00, 11.96it/s]

Classification Report for Task hatespeech_classes trained for 5 epochs
                  precision    recall  f1-score   support

               0  0.9699054170249355 0.9715762273901809 0.9707401032702238      1161
               1  0.6666666666666666 0.5714285714285714 0.6153846153846153        28
               2  0.0000000000000000 0.0000000000000000 0.0000000000000000         4
               3  0.5333333333333333 0.5714285714285714 0.5517241379310344        14
               4  0.0000000000000000 0.0000000000000000 0.0000000000000000         1
               5  0.1538461538461539 0.2000000000000000 0.1739130434782609        10
               6  0.6909090909090909 0.7307692307692307 0.7102803738317757        52

        accuracy                      0.9385826771653544      1270
       macro avg  0.4306658088257400 0.4350289430023649 0.4317203248422729      1270
    weighted avg  0.9367395722559195 0.9385826771653544 0.9375258873484790      1270




/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this

### 2 Epochs

In [18]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8418723545933512}

In [19]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8371149406524729}

In [20]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.3926597611174843}

### 3 Epochs

In [21]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8418723545933512}

In [27]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8325985296056062}

In [28]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4176057674898591}

### 4 Epochs

In [25]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8469622661091527}

In [27]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8325985296056062}

In [28]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4176057674898591}

### 5 Epochs

In [43]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8387301587301587}

In [44]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8107379458902904}

In [45]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4355893433350976}

## Loading

In [46]:
def load_model(dot_bin_file):
    multitask_model = MultitaskModel.create(
                                            model_name=model_name,
                                            model_type_dict={
                                                "offensive": transformers.AutoModelForSequenceClassification,
                                                "hatespeech": transformers.AutoModelForSequenceClassification,
                                                "hatespeech_classes": transformers.AutoModelForSequenceClassification,
                                            },
                                            model_config_dict={
                                                "offensive": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
                                                "hatespeech": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
                                                "hatespeech_classes": transformers.AutoConfig.from_pretrained(model_name, num_labels=7),
                                            },)
    model = torch.load(dot_bin_file)
    multitask_model.load_state_dict(model)
    return multitask_model

# Without Last Task

In [1]:
%cd /scratch/mt/ashapiro/Hate_Speech/Multitask_trial/

/scratch/mt/ashapiro/Hate_Speech/Multitask_trial


In [2]:
import numpy as np
import torch
import torch.nn as nn
import transformers
import nlp
import logging
from datasets import load_dataset
from model import * 
logging.basicConfig(level=logging.INFO)

## Preparing Data

In [3]:
task_names = ['offensive', 'hatespeech', 'hatespeech_classes']

In [4]:
dataset_dict = {
    "offensive": load_dataset("csv", data_files={'train': "Data/trainA_prepro.csv", 'test': "Data/testA_prepro.csv" } ),
    "hatespeech": load_dataset("csv", data_files={'train': "Data/trainB_prepro.csv", 'test': "Data/testB_prepro.csv" } ),
    "hatespeech_classes": load_dataset("csv", data_files={'train': "Data/trainC_prepro.csv", 'test': "Data/testC_prepro.csv" } ),
}

100%|██████████| 2/2 [00:00<00:00, 206.70it/s]


## Setting Model

In [5]:
model_name = "/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/"
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        "offensive": transformers.AutoModelForSequenceClassification,
        "hatespeech": transformers.AutoModelForSequenceClassification,
        "hatespeech_classes": transformers.AutoModelForSequenceClassification,
    },
    model_config_dict={
        "offensive": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "hatespeech": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "hatespeech_classes": transformers.AutoConfig.from_pretrained(model_name, num_labels=7),
    },
)

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [7]:
max_length = 512

def convert_to_features(example_batch):
    inputs = list(example_batch['text'])
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["labels"]
    return features

convert_func_dict = {
    "offensive": convert_to_features,
    "hatespeech": convert_to_features,
    "hatespeech_classes": convert_to_features,
}

In [8]:
columns_dict = {
    "offensive": ['input_ids', 'attention_mask', 'labels'],
    "hatespeech": ['input_ids', 'attention_mask', 'labels'],
    "hatespeech_classes": ['input_ids', 'attention_mask', 'labels'],
}

features_dict = {}
for task_name, dataset in dataset_dict.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
        features_dict[task_name][phase] = phase_dataset.map(
            convert_func_dict[task_name],
            batched=True,
            load_from_cache_file=False,
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        features_dict[task_name][phase].set_format(
            type="torch", 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

100%|██████████| 9/9 [00:03<00:00,  2.52ba/s]


offensive train 8887 8887
offensive train 8887 8887


100%|██████████| 2/2 [00:00<00:00,  3.92ba/s]


offensive test 1270 1270
offensive test 1270 1270


100%|██████████| 9/9 [00:03<00:00,  2.35ba/s]


...

hatespeech test 1270 1270
hatespeech test 1270 1270


100%|██████████| 9/9 [00:03<00:00,  2.35ba/s]


hatespeech_classes train 8887 8887
hatespeech_classes train 8887 8887


100%|██████████| 2/2 [00:00<00:00,  3.87ba/s]

hatespeech_classes test 1270 1270
hatespeech_classes test 1270 1270


In [9]:
eval_dataset = {
    task_name: dataset["test"] 
    for task_name, dataset in features_dict.items()
}

In [10]:
train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in features_dict.items()
}
args = transformers.TrainingArguments(
        output_dir="./models/multitask_model/3_epochs",
        overwrite_output_dir=True,
        learning_rate=2e-5,
        do_train=True,
        num_train_epochs=2,
        # Adjust batch size if this doesn't fit on the Colab GPU
        per_device_train_batch_size=16,  
        save_steps=3000,)

trainer = MultitaskTrainer(
    model=multitask_model,
    args=args,
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)

wandb: Currently logged in as: ahmadshapiro (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [11]:
trainer.train()

Iteration:  30%|██▉       | 500/1668 [04:31<11:57,  1.63it/s]

{"loss": 0.41617555819451807, "learning_rate": 1.7002398081534774e-05, "epoch": 0.2997601918465228, "step": 500}



Iteration:  60%|█████▉    | 1000/1668 [09:02<06:34,  1.69it/s]

{"loss": 0.2891820684103295, "learning_rate": 1.4004796163069546e-05, "epoch": 0.5995203836930456, "step": 1000}



Iteration:  90%|████████▉ | 1500/1668 [13:33<01:40,  1.68it/s]

...

{"loss": 0.13271094839542638, "learning_rate": 5.011990407673861e-06, "epoch": 1.498800959232614, "step": 2500}



Iteration:  80%|███████▉  | 1331/1668 [12:04<03:03,  1.84it/s]

{"loss": 0.11166536170669134, "learning_rate": 2.0143884892086333e-06, "epoch": 1.7985611510791366, "step": 3000}



Epoch: 100%|██████████| 2/2 [30:13<00:00, 906.95s/it]


TrainOutput(global_step=3336, training_loss=0.21281198457132977)

In [12]:
task_names = ['offensive','hatespeech','hatespeech_classes']

In [13]:
import datasets

In [14]:
f1 = datasets.load_metric('f1')

In [15]:
recall = datasets.load_metric('recall')

In [16]:
precision = datasets.load_metric('precision')

In [17]:
preds_dict = {}
for task_name in task_names:
    eval_dataloader = DataLoaderWithTaskname(
        task_name,
        trainer.get_eval_dataloader(eval_dataset=features_dict[task_name]["test"])
    )
    print(eval_dataloader.data_loader.collate_fn)
    preds_dict[task_name] = trainer._prediction_loop(
        eval_dataloader, 
        description=f"Test: {task_name}",
    )

Test: offensive:   1%|▏         | 2/159 [00:00<00:15, 10.40it/s]

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7ffef9fbb070>>


Test: hatespeech:   1%|▏         | 2/159 [00:00<00:13, 11.97it/s]

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7ffef9fbb070>>


Test: hatespeech_classes:   1%|▏         | 2/159 [00:00<00:13, 11.92it/s]

<bound method NLPDataCollator.collate_batch of <model.NLPDataCollator object at 0x7ffef9fbb070>>


Test: hatespeech_classes: 100%|██████████| 159/159 [00:13<00:00, 11.83it/s]


### 2 Epochs

In [18]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8418723545933512}

In [19]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8371149406524729}

In [20]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.3926597611174843}

### 3 Epochs

In [21]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8418723545933512}

In [27]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8325985296056062}

In [28]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4176057674898591}

### 4 Epochs

In [25]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8469622661091527}

In [27]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8325985296056062}

In [28]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4176057674898591}

### 5 Epochs

In [43]:
f1.compute(predictions=np.argmax(preds_dict["offensive"].predictions, axis=1),  references=preds_dict["offensive"].label_ids, average='macro' )

{'f1': 0.8387301587301587}

In [44]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech"].predictions, axis=1),  references=preds_dict["hatespeech"].label_ids, average='macro' )

{'f1': 0.8107379458902904}

In [45]:
f1.compute(predictions=np.argmax(preds_dict["hatespeech_classes"].predictions, axis=1),  references=preds_dict["hatespeech_classes"].label_ids, average='macro' )

{'f1': 0.4355893433350976}

## Loading

In [46]:
def load_model(dot_bin_file):
    multitask_model = MultitaskModel.create(
                                            model_name=model_name,
                                            model_type_dict={
                                                "offensive": transformers.AutoModelForSequenceClassification,
                                                "hatespeech": transformers.AutoModelForSequenceClassification,
                                                "hatespeech_classes": transformers.AutoModelForSequenceClassification,
                                            },
                                            model_config_dict={
                                                "offensive": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
                                                "hatespeech": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
                                                "hatespeech_classes": transformers.AutoConfig.from_pretrained(model_name, num_labels=7),
                                            },)
    model = torch.load(dot_bin_file)
    multitask_model.load_state_dict(model)
    return multitask_model